In [ ]:
import re
import string
from natasha import MorphVocab
from loguru import logger
import numpy as np
import pandas as pd
from razdel import tokenize
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
vectorized_news_df = pd.read_csv('reduced_news.csv', delimiter =',')



In [ ]:

category_counts = vectorized_news_df['tags'].value_counts()

min_count = 15000
# Отфильтровываем строки, где количество категорий больше или равно min_count
vectorized_news_df = vectorized_news_df[vectorized_news_df['tags'].isin(category_counts[category_counts > min_count].index)]

In [ ]:
vectorized_news_df['tags'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
test_df = vectorized_news_df.head(150)
test_df['target'] = labelencoder.fit_transform(test_df['tags'])
test_df

In [ ]:
X = test_df['text'].tolist()
Y = test_df['target'].tolist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
X_train

In [ ]:
from time import sleep

for i in tqdm(range(10)):
    sleep(3)

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)
from navec import Navec

# Создание объектов для сегментации и морфологического анализа
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

# Функция для лемматизации текста
def lemmatize_text(text):
    # Морфологический анализ сегментов
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    lemmas = []
    vectors = []
    for token in doc.tokens:
        if token.pos != 'PUNCT':
            # Получение леммы для каждого токена
            token.lemmatize(morph_vocab)
            lemmas.append(token.lemma)
            if token.lemma in emb:
                vector = emb[token.lemma]
                vectors.append(vector)
            else:
                vectors.append(np.zeros(300).astype(float))
    vectors = np.asarray(vectors)
    vectors = vectors.mean(axis=0)
    return vectors
# Пример использования функции для лемматизации текста
vectorized_news_df['vectors'] = [lemmatize_text(text) for text in tqdm(vectorized_news_df['text'])]
vectorized_news_df

In [ ]:
vectorized_news_df.to_csv('reduced_news_with_vectors.csv', encoding='utf-8')

# Обучение

In [1]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric, Dataset
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split

D:\VKR\researches\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vectorized_news_df = pd.read_csv('reduced_news.csv', delimiter =',')

In [3]:
category_counts = vectorized_news_df['tags'].value_counts()

min_count = 15000
# Отфильтровываем строки, где количество категорий больше или равно min_count
vectorized_news_df = vectorized_news_df[vectorized_news_df['tags'].isin(category_counts[category_counts > min_count].index)]

In [16]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
test_df = vectorized_news_df.head(150)
test_df['target'] = labelencoder.fit_transform(test_df['tags'])
test_df['tags']


C:\Users\y0urc\AppData\Local\Temp\ipykernel_21784\121829821.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['target'] = labelencoder.fit_transform(test_df['tags'])


0             Общество
1             Политика
2             Культура
3             Политика
4             Общество
            ...       
174    Наука и техника
175           Общество
176           Культура
177           Культура
178        СВО/Украина
Name: tags, Length: 150, dtype: object

In [5]:
X = test_df['text'].tolist()
Y = test_df['target'].tolist()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [17]:
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased-sentence', num_labels=9).to("cpu")
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
seq_len_train = [len(str(i).split()) for i in X_train]
seq_len_test = [len(str(i).split()) for i in X_test]
max_seq_len = max(max(seq_len_test), max(seq_len_train))
max_seq_len

377

In [19]:
tokens_train = tokenizer.batch_encode_plus(
    X_train,
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)
tokens_test = tokenizer.batch_encode_plus(
    X_test,
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)

In [20]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
    def __len__(self):
        return len(self.labels)
    
train_dataset = Data(tokens_train, y_train)
test_dataset = Data(tokens_test, y_test)

In [25]:
from sklearn.metrics import f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='macro')
    return {'F1': f1}

In [26]:
training_args = TrainingArguments(
    output_dir = './results', #Выходной каталог
    num_train_epochs = 3, #Кол-во эпох для обучения
    per_device_train_batch_size = 8, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 8, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=21)

In [27]:
trainer = Trainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = train_dataset,
                  compute_metrics = compute_metrics)

D:\VKR\researches\venv\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,2.025300,1.926534,0.147643
2,1.891800,1.814035,0.261807
3,1.824400,1.772558,0.326796


TrainOutput(global_step=39, training_loss=1.9138592451046674, metrics={'train_runtime': 2496.1188, 'train_samples_per_second': 0.12, 'train_steps_per_second': 0.016, 'total_flos': 58124473921800.0, 'train_loss': 1.9138592451046674, 'epoch': 3.0})